In [1]:
import os
from my_tools import *
from dependency import *
from instrumentation import *
from insert_patch import *
from program_synthesis import *
from get_heap_object import *
import argparse

In [ ]:
parser = argparse.ArgumentParser(description='file name')
parser.add_argument('f',type=str, help='file name')
args = parser.parse_args()
filename=args.f

In [ ]:
fuzz_cmd=[
    r'gcc -fpreprocessed -dD -E src.c | sed "/#/c\ " | sed "1d"',
    r"clang-12 -g -fsanitize=address,fuzzer src.c",
    r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>fuzzer_result"]



In [ ]:
def main(filename):
    os.system(f"gcc -fpreprocessed -dD -E {filename} | sed \"1d\" > {filename}")
    os.system(f"clang-12 -g -fsanitize=address,fuzzer {filename}")
    os.system(r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>fuzzer_result")
    os.system(f"cat {filename} | sed \"/#/c\ \" > dep_{filename} ")
    
    dep,error_feature = get_dependency("fuzzer_result")
    inst_filename = instrument(dep)
    
    os.system(f"clang-12 -g -fsanitize=address,fuzzer {inst_filename}")
    os.system(r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>inst_result")
    
    syn_inf = get_synthesis_inf(dep,"inst_result")
    # run 100 times to get more dynamic value
    for i in range(100):
        os.system(r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>inst_result")
        add_dynamic_value(syn_inf,"inst_result")
    
    # def a func
    for err_inf,err_dep,err_fea in zip(syn_inf,dep,error_feature):
        patch_cand = synthesis().synthesis(err)
        filelist = read_file(filename)
        filelist,patch = insert_tmp_var(filelist,patch_cand)
        for k,v in patch.items():
            #TODO
            o,otype,line=get_error_object(err_dep,k,v)
            if o == 0:
                continue
            filelist,o=insert_heap_object(filelist,o,otype,line)
            free=cur_patch(patch,o)
            for ret in v["ret"]:
                patch_line=int(ret["coord"].split(":")[1])
                filelist=insert_patch(filelist,patch_line,free)
                write_file("fixed_"+filename,filelist)
                os.system(f"clang-12 -g -fsanitize=address,fuzzer fixed_{filename}")
                os.system(r"./a.out -max_total_time=5 -keep_seed=1 -max_len=2 2>cur_fuzzer_result")
                cur_err=get_error_feature(get_fuzzer_result("cur_fuzzer_result"))
                if error_feature == cur_err:
                    #repair a part
                for err in cur_err:
                    if err not in error_featur:
                        #new error
                if err_fea not in cur _err:
                    #repaired
                        

In [ ]:
file=open("inst_result",'r')
inf=file.read()
for i in inf.split("\n"):
    if "instrument:" in i :
        print(i)

In [ ]:
{'func': {'patch': 'tmp_a==5',
   'ret': [{'coord': 'dep__src.c:12:9'}, {'coord': 'dep__src.c:12:16'}]}}

In [4]:
syn_inf=get_synthesis_inf(dep,"inst_result")

In [5]:
syn_inf

[{'error': [0, 1],
  'var': {'func': [{'state': 'input',
     'type': 'int',
     'name': 'a',
     'coord': 'dep__src.c:5:14',
     'value': [0, 5],
     'ret': [{'coord': 'dep__src.c:10:9'},
      {'coord': 'dep__src.c:12:9'},
      {'coord': 'dep__src.c:12:16'}]}]}}]

In [ ]:
add_dynamic_value(syn_inf,"inst_result")

In [ ]:
syn_inf

In [ ]:
get_dynamic_value("inst_result")

In [6]:
def get_error_object(dep,func):
    if dep[func]["object"][1]==dep["object"][1]:
        return dep[func]["object"]
    else:
        o = dep["object"][0]
        otype = dep["object"][1]
        line = dep[func]["object"][2]
        if "->" in o:
            namelist=o.split("->")
            if "." in namelist[0]:
                name=dep[func]["object"][0]+"."+".".join(namelist[0].split(".")[1:])+"->"+"->".join(namelist[1:])
            else:
                name=dep[func]["object"][0]+"->"+"->".join(namelist[1:])
        elif "." in o:
            namelist=o.split(".")
            name=dep[func]["object"][0]+"."+".".join(namelist[1:])
        else:
            return 0,0,0
        return (name,otype,line)
    

In [2]:
dep = get_dependency("fuzzer_result")

In [3]:
dep

[{'func': {'dep': [{'name': 'a',
     'state': 'input',
     'type': 'int',
     'coord': 'dep__src.c:5:14'},
    {'name': 'b', 'state': 'var', 'type': 'int*', 'coord': 'dep__src.c:7:13'}],
   'ret': [{'coord': 'dep__src.c:10:9'},
    {'coord': 'dep__src.c:12:9'},
    {'coord': 'dep__src.c:12:16'}],
   'object': ('p', 'int*', 6)},
  'error_object': ('p', 'int*', 6)}]

In [ ]:
data={
        "error":[1,1,1,1,1,1,1,1,1,1],
        "var":{"func":[
            {
                "state":"input",
                "type":"int",
                "name":"a",
                "coord":2,
                "value":[2,2,6,6,6,5,4,3,4,10],
                "ret":[]
            },
            {
                "state":"input",
                "type":"int",
                "name":"b",
                "coord":2,
                "value":[0,1,-2,-1,0,2,4,6,6,1],
                "ret":[]
            },
            {
                "state":"var",
                "type":"int",
                "name":"c",
                "coord":6,
                "value":[2,3,4,5,6,7,8,9,10,11],
                "ret":[]
            }
        ]}
    
}